# Metro Interstate Traffic Volume

**Data Set Information:**

Hourly Interstate 94 Westbound traffic volume for MN DoT ATR station 301, roughly midway between Minneapolis and St Paul, MN. Hourly weather features and holidays included for impacts on traffic volume.


**Attribute Information:**

**- holiday:** Categorical US National holidays plus regional holiday, Minnesota State Fair;<br>
**- temp:** Numeric Average temp in kelvin;<br>
**- rain_1h:** Numeric Amount in mm of rain that occurred in the hour;<br>
**- snow_1h:** Numeric Amount in mm of snow that occurred in the hour;<br>
**- clouds_all:** Numeric Percentage of cloud cover;<br>
**- weather_main:** Categorical Short textual description of the current weather;<br>
**- weather_description:** Categorical Longer textual description of the current weather;<br>
**- date_time:** DateTime Hour of the data collected in local CST time;<br>
**- traffic_volume:** Numeric Hourly I-94 ATR 301 reported westbound traffic volume.

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics

import sys
sys.path.insert(1, '../RegressionAlgorithms/')
import linearRegression
from knn import *

### Get the Data

In [ ]:
data = pd.read_csv('MetroInterstateTrafficVolume.csv')

In [ ]:
data

### Basic Data Information

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.describe(include = 'object')

### Check missing values

In [ ]:
print(data.isnull().sum(axis=0))

### Exploratory Data Analysis

**Traffic Volume**

*Histogram of Traffic Volume distribution*

In [ ]:
fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
bins = np.arange(0, 7500, 250).tolist()
data['traffic_volume'].hist(bins=bins)
plt.xticks(bins)
plt.xlabel('traffic_volume')

**Traffic Volume vs Holiday**

*Check holidays included in the dataset*

In [ ]:
data['holiday'].unique()

*Box plot of Traffic Volume vs Holiday distribution*

In [ ]:
plt.figure(figsize=(20, 8))
sns.boxplot(x=data['holiday'], y=data['traffic_volume'])
plt.show()

*Distribution only with the holidays*

In [ ]:
data_holidays = data.loc[(data['holiday'] != 'None')]
data_holidays.index = np.arange(1, len(data_holidays) + 1)
data_holidays

*Box plot of Traffic Volume vs Holiday distribution (only holidays included)*

In [ ]:
plt.figure(figsize=(20, 8))
sns.boxplot(x=data_holidays['holiday'], y=data_holidays['traffic_volume'])
plt.show()

**Traffic Volume vs Temperature**

*Plot of Traffic Volume vs Temperature distribution*

In [ ]:
fig = sns.jointplot(x=data['temp'], y=data['traffic_volume'], kind='reg')

*Removing outliers*

In [ ]:
outliers = data[(data['temp'] <= 50)]
data = data.drop(outliers.index)
data.index = np.arange(1, len(data) + 1)
outliers

*Plot of Traffic Volume vs Temperature distribution (without outliers)*

In [ ]:
fig = sns.jointplot(x=data['temp'], y=data['traffic_volume'], kind='reg')

**Traffic Volume vs Rain**

*Plot of Traffic Volume vs Rain distribution*

In [ ]:
fig = plt.figure(figsize = (25,15))
ax1 = fig.add_subplot(2,3,1)
ax1.scatter(data['rain_1h'], data['traffic_volume'])

*Removing outliers*

In [ ]:
outliers = data[(data['rain_1h'] >= 1000)]
data = data.drop(outliers.index)
data.index = np.arange(1, len(data) + 1)
outliers

*Plot of Traffic Volume vs Rain distribution (without outliers)*

In [ ]:
fig = sns.jointplot(x=data['rain_1h'], y=data['traffic_volume'], kind='reg')

*Distribution only with rainy days*

In [ ]:
data_rainy = data.loc[(data['rain_1h'] > 0)]
#data_rainy = data.loc[(data['weather_main'] == "Rain")]
data_rainy.index = np.arange(1, len(data_rainy) + 1)
data_rainy

*Plot of Traffic Volume vs Rain distribution (only rainy days included)*

In [ ]:
fig = sns.jointplot(x=data_rainy['rain_1h'], y=data_rainy['traffic_volume'], kind='reg')

**Traffic Volume vs Snow**

*Plot of Traffic Volume vs Snow distribution*

In [ ]:
fig = plt.figure(figsize = (25,15))
ax1 = fig.add_subplot(2,3,1)
ax1.scatter(data['snow_1h'], data['traffic_volume'])

*Distribution only with snowy days*

In [ ]:
data_snowy = data.loc[(data['snow_1h'] > 0)]
#data_snowy = data.loc[(data['weather_main'] == "Snow")]
data_snowy.index = np.arange(1, len(data_snowy) + 1)
data_snowy

*Plot of Traffic Volume vs Snow distribution (only snowy days included)*

In [ ]:
fig = sns.jointplot(data_snowy['snow_1h'], data_snowy['traffic_volume'], kind='reg')

**Traffic Volume vs Cloud cover**

*Plot of Traffic Volume vs Cloud cover distribution*

In [ ]:
fig = plt.figure(figsize = (25,15))

ax1 = fig.add_subplot(2,3,1)
ax1.scatter(data['clouds_all'], data['traffic_volume'])

In [ ]:
#data['clouds_all'].unique()
#x = data['clouds_all'].value_counts()
#y = list(x[:10].index)

#plt.figure(figsize=(20, 8))
#sns.boxplot(data['clouds_all'], data['traffic_volume'])
#plt.show()

**Traffic Volume vs Current weather**

*Box plot of Traffic Volume vs Current weather distribution*

In [ ]:
plt.figure(figsize=(20, 8))
sns.boxplot(x=data['weather_main'], y=data['traffic_volume'])
plt.show()

**Traffic Volume vs Date time**

*Separation of the date elements*

In [ ]:
data

In [ ]:
data[['year','month','day','hour','minutes','seconds']] = data['date_time'].str.extract(r'(\d+)-(\d+)-(\d+)\s*(\d+):(\d+):(\d+)', expand=True)
data = data.drop(['date_time'], axis=1)
data[['year','month','day','hour','minutes','seconds']] = data[['year','month','day','hour','minutes','seconds']].astype(float)

*Dataset with new labels*

In [ ]:
data

*Box plot of Traffic Volume vs Year*

In [ ]:
plt.figure(figsize=(20, 8))
sns.boxplot(x=data['year'], y=data['traffic_volume'])
plt.show()

*Box plot of Traffic Volume vs Month*

In [ ]:
plt.figure(figsize=(20, 8))
sns.boxplot(x=data['month'], y=data['traffic_volume'])
plt.show()

*Box plot of Traffic Volume vs Hour*

In [ ]:
plt.figure(figsize=(20, 8))
sns.boxplot(x=data['hour'], y=data['traffic_volume'])
plt.show()

In [ ]:
data

## Preprocessing

*Drop minutes and second columns*

In [ ]:
data.drop(['minutes', 'seconds'], axis=1)

*Preprocess Non Ordinal Data*

In [ ]:
one_hot = pd.get_dummies(data["weather_main"])
data = data.drop("weather_main",axis = 1)
data = data.join(one_hot.astype(float))

In [ ]:
one_hot = pd.get_dummies(data["weather_description"])
data = data.drop("weather_description",axis = 1)
data = data.join(one_hot.astype(float))

In [ ]:
one_hot = pd.get_dummies(data["holiday"])
data = data.drop("holiday",axis = 1)
data = data.join(one_hot.astype(float))

In [ ]:
data.info()

**Data Preparation**

In [ ]:
X = data.drop('traffic_volume', axis=1)
y = data['traffic_volume']

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(X)
X_scaled = scaler.transform(X)

#split the data in attributes and class as well as training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

## Regression Tasks

*Regression Algorithms from Sklearn*

### Linear Regression

In [ ]:
model = linear_model.LinearRegression().fit(X_train, y_train)

# Make predictions using the testing set
y_pred1 = model.predict(X_test)

# The coefficients
print('Coefficients: \n', model.coef_)
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred1))

print('-------------------------------------------')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred1))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred1))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred1)))

### KNN Regression

In [ ]:
model = KNeighborsRegressor(n_neighbors=3).fit(X_train, y_train)

# Make predictions using the testing set
y_pred = model.predict(X_test)

# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

print('-------------------------------------------')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

*Our regression algorithms*

### Linear Regression Function

In [ ]:
stop

In [ ]:
%load_ext autoreload
%autoreload 2

try:
    del X_train['bias']
except:
    print('no bias to remove X_train')    
try:
    del X_test['bias']
except:
    print('no bias to remove X_test')
try:
    del X['bias']
except:
    print('no bias to remove X')


import linearRegressionNumpy

    
#print('\nMatrix Solution for Comparison:')
#weightsMatrix = linearRegressionNumpy.matrixSolution(X_train, y_train)
#yPredMatrix = linearRegressionNumpy.predictLinearRegression(X_test, weightsMatrix)
#print('weights = ', weightsMatrix)


print('\n Now our own iterative regression algorithm:')    
alpha = [] #[500, 500, 500, 500]
alphaMethod = 'const'
mu = 1
convCritList = [1e5, 1e4, 1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
print('epsilon       | sum total error:   | sum relative error:  | iterations | Rsquare   ')
for convergenceCriterion in convCritList:

    weights, score, iterations = linearRegressionNumpy.linearRegression(X_train, y_train, alpha = alpha, mu = mu, 
                                                        convergenceCriterion = convergenceCriterion, lossFunction = 'MSE', 
                                                        alphaMethod = alphaMethod, printOutput = False)
    yPred2 = linearRegressionNumpy.predictLinearRegression(X_test, weights)



    print('{:13E} | {:19}| {:21}| {:11}| {:10}'.format(convergenceCriterion, 
                                        str(np.sum(yPred2-y_pred1)), 
                                        str(np.sum((yPred2-y_pred1)/y_pred1)),
                                        str(iterations),
                                        str(r2_score(y_test, yPred2))))
    #print('sum total error: us iterative vs. scikit = ', np.sum(yPred2-y_pred1))
    #print('sum relative error: us iterative vs. scikit = ', np.sum((yPred2-y_pred1)/y_pred1))

print('yPred = ', yPred2)
print('weights = ', weights)
print('score = ', score)   
    
#print('\n\n\n\n\n')

plt.plot(y_pred1, label = 'scikit')
plt.legend()
plt.show()
plt.plot(yPred2, label = 'us iterative solution')
plt.legend()
plt.show()
plt.plot(yPredMatrix, label = 'us matrix solution')
plt.legend()
plt.show()

plt.plot(yPred2-y_pred1, label = 'total error, us iterative vs. scikit')
plt.legend()
plt.show()
plt.plot(yPredMatrix-y_pred1, label = 'total error, us matrix vs. scikit')
plt.legend()
plt.show()
plt.plot(yPred2-yPredMatrix, label = 'total error, us iterative vs. us matrix')
plt.legend()
plt.show()

plt.plot((yPred2-y_pred1)/y_pred1, label = 'relative error, us iterative vs. scikit')
plt.legend()
plt.show()
plt.plot((yPredMatrix-y_pred1)/y_pred1, label = 'relative error, us matrix vs. scikit')
plt.legend()
plt.show()
plt.plot((yPred2-yPredMatrix)/yPredMatrix, label = 'relative error, us vs. us matrix')
plt.legend()
plt.show()

### KNN

**Dictionary creation to apply the mathematical functions of the algorithm**

In [ ]:
dictionary = data.to_dict('records')

In [ ]:
dictionary

In [ ]:
len(dictionary)

**Forecasting instances**

In [ ]:
y_test

**Algorithm parameters**

In [ ]:
mode = 1 # 1 = KNeighbors; 2 = RadiusNeighbors
n_neighbours = 5
distance_function = 1 # 1 = Euclidean Distance; 2 = Manhattan Distance
radius = 0 # 0 indicates no radius
label = 'traffic_volume'
features = ['temp']

**Algorithm initialization**

In [ ]:
knn = KNN(dictionary, label, features, mode, n_neighbours, distance_function, radius)

**Execution of the algorithm (forecasting)**

In [ ]:
results = []

for x in y_test.index:
    #print(x)
    target = dictionary[x-1]
    #print(target)
    result = knn.run(target)
    #print(result)
    results.append(result)

**Predictions**

In [ ]:
predictions = pd.Series(results,index=y_test.index)

In [ ]:
predictions

**Evaluation metrics**

In [ ]:
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, predictions))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))